### Домашнее задание

In [175]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [176]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0).astype(int)
    

    return hit_rate

In [177]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [178]:
def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [179]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
#-----------------------------------------------------------------------------------------------
# исправил строку...    
# было
    flags = np.isin(recommended_list, bought_list)
# стало
    flags = np.isin(recommended_list[:k], bought_list)
#-----------------------------------------------------------------------------------------------
        
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        if flags[i-1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
    result = sum_ / len(flags)
    
    return result

In [180]:
def ndcg_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list[:k], bought_list)
        
    if sum(flags) == 0:
        return 0
    
    dcg = 0
    dcg_ideal = 0
    
    for i in range(k):
        if (i+1 <= 2):
            coef = 1
        else:
            coef = 1 / np.log2(i+1)
        
        dcg_ideal += 1 * coef
        
        if flags[i]:
            dcg += 1 * coef
    
    dcg /= k
    dcg_ideal /= k
    result = dcg / dcg_ideal
   
    return result

In [181]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    ranks = 0.
    
    for i, item_rec in enumerate(recommended_list[:k]):
        for item_bought in bought_list:
    
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    
    return ranks / len(recommended_list[:k])

### Задание 1

- Сравните метрики hit_rate@k, precision@k.   
- Какую метрику использовать предпочтительно и почему?  
- Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


In [182]:
# Пример данных и метрик - 2 случая
recommended_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

bought_list_1 = [ 3,  9, 26, 44, 72]
bought_list_2 = [12, 17, 26, 44, 72]

hr_1 = hit_rate_at_k(recommended_list, bought_list_1)
hr_2 = hit_rate_at_k(recommended_list, bought_list_2)

pr_1 = precision_at_k(recommended_list, bought_list_1)
pr_2 = precision_at_k(recommended_list, bought_list_2)

print("recommended_list =", recommended_list)
print("bought_list_1 =", bought_list_1)
print("bought_list_2 =", bought_list_2)

print()
# print(f'hit_rate@k_1 = {hr_1};\t hit_rate@k_2 = {hr_2}')
# print(f'precision@k_1 = {pr_1*100}%;\t precision@k_2 = {pr_2*100}%')
print(f'hit_rate@k_1 = {hr_1};\t precision@k_1 = {pr_1*100}%')
print(f'hit_rate@k_2 = {hr_2};\t precision@k_2 = {pr_2*100}%')

recommended_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
bought_list_1 = [3, 9, 26, 44, 72]
bought_list_2 = [12, 17, 26, 44, 72]

hit_rate@k_1 = 1;	 precision@k_1 = 20.0%
hit_rate@k_2 = 0;	 precision@k_2 = 0.0%


Метрика **Нit_rate** просто показывает было ли взимодействие пользователя и рекомендованных (k) товаров. Принимает целые значение 0 и 1. Где 0 - не было взаимодействия и 1 - было.

Метрика **Precision** показывает долю, какая часть рекомендованных (k) товаров оказалось релевантной. Принимает значение вещественные значения от 0 до 1.

Предпочтительный использовать метрику Precision, т.к. она по сути уже включает в себя Hite_rate и является более информативным показателем.  
Precision, не равное нулю, означает, что взаимодействие было.  
А фактическое значение Precision показывает точность рекомендации.

**Где можно использовать метрику Hit_rate**

1. **Любые выборы** (президента, различных органов власти и т.д.)

   Клиенты - избиратели, товар - кандидаты.  
   Человек участвовал в выборах - 1; Не участвовал - 0  
        
      
2. **Предложение банковских продуктов**: кредитов, вкладов, карт и т.д.  
   
   Клиенты - люди; Товар - продукты банка  
   Если человек воспользовался хотя бы одним продуктом - 1 (стал клиентом банка); Не воспользовался - 0  
      
      
3. **Продажа туров туристическими компаниями**

    Клиенты - люди; Товар - различные варианты туров  
    Человек купил тур - 1; Не купил - 0

### Задание 2

- В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм?
- Какую функцию можно использовать вместо логарифма?
- Приведите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.

1. В метрике NDCG используется логарифм чтобы взвесить значения, представленные в списке. Логарифм придает значениям в начале списка бОльший вес, чем значениям в конце. Рост функции Log замедляется, поэтому веса меняются очень плавно.


2. Можно использовать степенную функцию с показателем (1/N), т.е. с показателдем меньше единицы (или корень N степени)


3. Примеры метрик и подходов c логарифмами:

    - Логистическая функция потерь  
    - Критерий информативности - энтропия Шеннона  
    - Коэффициент доверия классификатору  
    - Гомогенность, Полнота, V-мера (для кластеров и классов)  
    - Взаимная информация

### Задание 3

Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

1. **CVR (conversion rate)** - коэффициент конверсии.

    Отношение полученных активных действий (лидов, заказов, транзакций и пр.) к общему количеству аудитории.  
    Интеракции - активные действия пользователей.
    
    
2. **CTR (click throught rate)** - кликабельность.

    Отношение числа кликов по рекламному объявлению/баннеру к числу показов.  
    Интеракция - клик по рекламному объявлению.
    
    
3. **OR (Open-Rate)** - открываемость e-mail сообщений

    Отражает долю писем, открытых получателями в общей массе отправленных писем. Показывает, насколько удачной была рассылка.  
    Интеракция  - окрытие письма с сообщением.

### Задание 4

boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  

Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

**Пример выполнения 4го задания**

MSE на  
$y = [0, 2]$  
$y_{pred} = [1, 2]$

MSE = $0.5*((0 - 1)^2 + (2-2)^2) = 0.5$ 

In [183]:
import numpy as np
((np.array([0, 2]) - np.array([1, 2]))**2).mean()

0.5

In [184]:
boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3] 

#### PR@8

In [185]:
# [1, 3, 5, 7, 9, 11]  
# [2, 5, 7, 4, 11, 9, 8, 10, 12, 3] 

PR_8 = (0 + 0 + 1 + 1 + 1 + 1 + 0 + 0) / (1 * 8)
PR_8

0.5

In [186]:
precision_at_k(recommended, boughted, 8)

0.5

#### REC@8

In [187]:
# [1, 3, 5, 7, 9, 11]  
# [2, 5, 7, 4, 11, 9, 8, 10, 12, 3] 

REC_8 = (0 + 0 + 1 + 1 + 1 + 1 + 0 + 0) / (1 * 6)
REC_8

0.6666666666666666

In [188]:
recall_at_k(recommended, boughted, 8)

0.6666666666666666

#### AP@8

In [189]:
# [1, 3, 5, 7, 9, 11]  
# [2, 5, 7, 4, 11, 9, 8, 10, 12, 3] 

ap_8 = ( 0 * 0/1 + 1 * 1/2 + 1 * 2/3 + 0 * 2/4 + 1 * 3/5 + 1 * 4/6 + 0 * 4/7 + 0 * 4/8) / 8 
ap_8

0.30416666666666664

In [190]:
ap_k(recommended, boughted, 8)

0.30416666666666664

#### NDCG@8

In [191]:
# [1, 3, 5, 7, 9, 11]  
# [2, 5, 7, 4, 11, 9, 8, 10, 12, 3] 

DCG = (0/1 + 1/np.log2(2) + 1/np.log2(3) + 0/np.log2(4) + 1/np.log2(5) + 1/np.log2(6) + 0/np.log2(7) + 0/np.log2(8)) / 8
idealDCG = (1/1 + 1/np.log2(2) + 1/np.log2(3) + 1/np.log2(4) + 1/np.log2(5) + 1/np.log2(6) + 1/np.log2(7) + 1/np.log2(8)) / 8

print(f'\tDCG      = {DCG}\n\tidealDCG = {idealDCG}\n\tNDCG@8   = {DCG / idealDCG}')

	DCG      = 0.30605738985992403
	idealDCG = 0.5797499549150934
	NDCG@8   = 0.5279127445637272


In [192]:
ndcg_k(recommended, boughted, 8)

0.5279127445637272

#### MRR@8

In [193]:
# [1, 3, 5, 7, 9, 11]  
# [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

MRR_8 = (0 + 1/2 + 1/3 + 0 + 1/5 + 1/6 + 0 + 0) / 8
MRR_8 

0.15

In [194]:
reciprocal_rank(recommended, boughted, 8)

0.15